In [1]:
from sklearn.feature_extraction import DictVectorizer
import pandas as pd

In [2]:
kor_begin = 44032
kor_end = 55203
chosung_base = 588
jungsung_base = 28
jaum_begin = 12593
jaum_end = 12622
moum_begin = 12623
moum_end = 12643

chosung_list = [ 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
        'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
        'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
        'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
        'ㅡ', 'ㅢ', 'ㅣ']

jongsung_list = [
    ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
        'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
        'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
        'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
              'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
              'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
              'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

def compose(chosung, jungsung, jongsung):
    char = chr(
        kor_begin +
        chosung_base * chosung_list.index(chosung) +
        jungsung_base * jungsung_list.index(jungsung) +
        jongsung_list.index(jongsung)
    )
    return char

def decompose(c):
    if not character_is_korean(c):
        return None
    i = ord(c)
    if (jaum_begin <= i <= jaum_end):
        return (c, ' ', ' ')
    if (moum_begin <= i <= moum_end):
        return (' ', c, ' ')

    # decomposition rule
    i -= kor_begin
    cho  = i // chosung_base
    jung = ( i - cho * chosung_base ) // jungsung_base 
    jong = ( i - cho * chosung_base - jung * jungsung_base )    
    return (chosung_list[cho], jungsung_list[jung], jongsung_list[jong])

def character_is_korean(c):
    i = ord(c)
    return ((kor_begin <= i <= kor_end) or
            (jaum_begin <= i <= jaum_end) or
            (moum_begin <= i <= moum_end))

In [3]:
def levenshtein(s1, s2, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

In [4]:
def jamo_levenshtein(s1, s2):
    if len(s1) < len(s2):
        return jamo_levenshtein(s2, s1)

    if len(s2) == 0:
        return len(s1)
    
    def get_jamo_cost(c1, c2):
        if c1 == c2:
            return 0
        jamo1 = decompose(c1)
        jamo2 = decompose(c2)
        if jamo1 is None or jamo2 is None:
            return 1
        return levenshtein(decompose(c1), decompose(c2)) / 3

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + get_jamo_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [5]:
jamo_levenshtein('가나다-','나당라')

2.333333333333333

In [6]:
pwd

'C:\\Users\\admin\\Downloads\\ken\\kcc'

In [14]:
hotel = pd.read_csv('./hotel_csv.csv', encoding='cp949', engine='python')
hotel.columns = ['item', 'size', 'unit']
hotel_ = hotel.dropna(subset=['item', 'unit'], how='any')
wonju = pd.read_csv('./wonju.csv', encoding='cp949',engine='python')
wonju.columns = ['item', 'size', 'unit', 'amount']
wonju_ = wonju.dropna(subset=['item', 'unit'], how='any')
wonju_ = wonju_.reset_index(drop=True)
hotel_ = hotel_.reset_index(drop=True)

def custom_lev(col1, col2):
    w_item = wonju_[col1].tolist()
    h_item = hotel_[col2].tolist()
    all_dict = {}
    for i in w_item:
        all_dict[i]= [5000000012351518 , i, 0 ]
        for j in h_item :
            
            distance = jamo_levenshtein(str(i), str(j))
            accuracy = jamo_levenshtein(str(i), str(j))
            if distance < all_dict[i][0]:
                all_dict[i] = [distance, j, accuracy]
    return all_dict

In [15]:
arrr = custom_lev('item', 'item')

only_item = wonju_.copy()

only_item['items_dist'] = wonju_['item'].map(lambda x : arrr[x][0])
only_item['items_min_name'] = wonju_['item'].map(lambda x : arrr[x][1])
only_item['lev_dist_accuracy'] = wonju_['item'].map(lambda x : round(arrr[x][2],2))

In [20]:
only_item.sort_values(by='items_dist').head(50)

,item,size,unit,amount,items_dist,items_min_name,lev_dist_accuracy
116,창문틀 주위 충전,모르타르 충전,M,56.000,0.000000,창문틀 주위 충전,0.00
35,콘크리트벽돌,"콘크리트벽돌, 190*57*90mm, 강원, C종2급",매,17418.000,0.000000,콘크리트벽돌,0.00
79,오픈트랜치,"양면, L-25*25*3t 아연도금",M,20.000,0.000000,오픈트랜치,0.00
37,벽돌 운반,"인력, 2층",천매,8.130,0.000000,벽돌 운반,0.00
38,0.5B 벽돌쌓기,3.6m 이하,M2,24.000,0.000000,0.5B 벽돌쌓기,0.00
39,1.0B 벽돌쌓기,3.6m 이하,M2,100.000,0.000000,1.0B 벽돌쌓기,0.00
163,시멘트,건재상,포,948.000,0.000000,시멘트,0.00
162,모래,"모래, 도착도",M3,77.000,0.000000,모래,0.00
142,선홈통-스텐레스파이프-설치,∮127.0*1.2mm,M,43.000,0.000000,선홈통-스텐레스파이프-설치,0.00
90,콘크리트면 정리,NaN,M2,167.000,0.000000,콘크리트면 정리,0.00


In [23]:
#only_item.to_excel('./only_itemname_morepheme_v1_0507.xlsx', index=False, encoding='cp949')

In [25]:
wonju_['item']

0                  컨테이너형 가설건축물 - 창고.
1                 컨테이너형 가설건축물 - 사무실.
2                             평판재하시험
3                           조립식가설실험실
4                             수평 규준틀
5                            구조부 먹매김
6                   강관비계(쌍줄) 설치 및 해체
7                      강관 조립말비계(이동식)
8                      강관동바리 설치 및 해체
9                     시스템동바리 설치 및 해체
10                     건축물 보양 - 콘크리트
11                           건축물현장정리
12                            터파기/토사
13                  되메우기/토사, 두께 30cm
14                    토사 운반/단지외 10km
15            발포폴리스티렌(슬래브 위 깔기 - 바닥)
16                         방습필름 - 바닥
17                            잡석깔기지정
18             콘크리트 펌프차 타설(무근, 진동기無)
19               콘크리트 펌프차 타설(매트기초 등)
20     콘크리트 펌프차 타설(벽,기둥,슬래브 등) [PIT]
21     콘크리트 펌프차 타설(벽,기둥,슬래브 등) [01F]
22     콘크리트 펌프차 타설(벽,기둥,슬래브 등) [02F]
23     콘크리트 펌프차 타설(벽,기둥,슬래브 등) [RF1]
24     콘크리트 펌프차 타설(벽,기둥,슬래브 등) [PH1]
25                       유로폼 설치 및 해체
26                       유로폼 설치 및 해체
2

# NLP 적용 0507

In [26]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
import warnings
warnings.filterwarnings('ignore')

In [27]:
twitter = Twitter()
word_dic = {}
lines = wonju_['item']
for line in lines :
    malist = twitter.pos(line)
    for word in malist :
        if word[1] == "Noun":
            if not (word[0] in word_dic):
                word_dic[word[0]] = 0
            word_dic[word[0]] += 1

In [29]:
word_dic

{'컨테이너': 2,
 '가설': 3,
 '건축물': 4,
 '창고': 1,
 '사무실': 1,
 '평판': 1,
 '시험': 6,
 '조립': 3,
 '실험실': 1,
 '수평': 1,
 '규준': 1,
 '틀': 3,
 '조부': 1,
 '먹매김': 1,
 '강관': 2,
 '비계': 2,
 '쌍줄': 1,
 '설치': 24,
 '및': 8,
 '해체': 7,
 '말': 1,
 '이동식': 1,
 '강': 1,
 '관동': 1,
 '바리': 1,
 '시스템': 1,
 '동바리': 1,
 '보양': 1,
 '콘크리트': 16,
 '현장': 3,
 '정리': 2,
 '터': 1,
 '파기': 1,
 '토사': 3,
 '두께': 1,
 '운반': 6,
 '단지': 1,
 '외': 1,
 '발포': 3,
 '폴리스티렌': 3,
 '슬래브': 9,
 '위': 2,
 '바닥': 6,
 '방습': 2,
 '필름': 2,
 '잡석': 1,
 '지정': 1,
 '펌프': 7,
 '차': 7,
 '설': 9,
 '무근': 1,
 '진동기': 2,
 '매트': 1,
 '기초': 1,
 '등': 6,
 '벽': 8,
 '기둥': 5,
 '유로폼': 2,
 '합판': 2,
 '거푸집': 2,
 '지붕': 4,
 '노임': 2,
 '할증': 2,
 '엔진': 1,
 '인력': 1,
 '유': 1,
 '일반': 1,
 '철근': 3,
 '보통': 2,
 '가공': 1,
 '철강': 1,
 '경량': 4,
 '방재': 2,
 '벽돌': 7,
 '프라이머': 3,
 '바름': 8,
 '시멘트': 6,
 '액체': 2,
 '방수': 6,
 '우레': 2,
 '포함': 5,
 '타일': 4,
 '코너': 1,
 '비드': 2,
 '바탕': 5,
 '도': 2,
 '기질': 2,
 '압착': 3,
 '압': 3,
 '포슬린': 1,
 '화강석': 7,
 '재료': 2,
 '분리': 2,
 '습': 6,
 '물': 3,
 '갈기': 3,
 '겁돌': 1,
 '버너': 3,
 '자재': 3,
 

In [30]:
keys = sorted(word_dic.items(), key=lambda x:x[1], reverse = True)
for word, count in keys[:50]:
    print('{0}({1})'.format(word, count), end="")
print()

설치(24)견적(22)콘크리트(16)식별(10)번호(10)슬래브(9)설(9)및(8)벽(8)바름(8)물품(8)해체(7)펌프(7)차(7)벽돌(7)화강석(7)스테인리스(7)모르타르(7)시험(6)운반(6)바닥(6)등(6)시멘트(6)방수(6)습(6)핸드(6)레일(6)알루미늄(6)기둥(5)포함(5)바탕(5)유리(5)건축물(4)지붕(4)경량(4)타일(4)형(4)실(4)재(4)투명(4)비(4)철근콘크리트(4)용봉강(4)가설(3)조립(3)틀(3)현장(3)토사(3)발포(3)폴리스티렌(3)


In [31]:
from gensim.models import word2vec

twitter = Twitter()
results = []
lines = wonju_['item']
for line in lines :
    malist = twitter.pos(line, norm=True, stem=True)
    r = []
    for word in malist :
        # 어미/조사/구두점 등은 대상에서 제외
        if not word[1] in ['Josa', "Eomi", "Punctuation"]:
            r.append(word[0])
        rl = (" ".join(r)).strip()
        results.append(rl)
        #print(rl)

In [42]:
names = wonju_['item']
for i in names[:10]:
    print(i)

컨테이너형 가설건축물 - 창고.
컨테이너형 가설건축물 - 사무실.
평판재하시험
조립식가설실험실
수평 규준틀
구조부 먹매김
강관비계(쌍줄) 설치 및 해체
강관 조립말비계(이동식)
강관동바리 설치 및 해체
시스템동바리 설치 및 해체


In [43]:
twitter.pos(names[0], norm=True, stem=True)

[('컨테이너', 'Noun'),
 ('형', 'Suffix'),
 ('가설', 'Noun'),
 ('건축물', 'Noun'),
 ('-', 'Punctuation'),
 ('창고', 'Noun'),
 ('.', 'Punctuation')]

In [34]:
# 파일로 출력하기
wakati_file = 'wonju.wakati'
with open(wakati_file, 'w', encoding = 'utf-8') as fp:
    fp.write('\n'.join(results))
    
# Word2Vec 모델 만들기
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data,
                         size = 200, window=10, hs=1, min_count=2, sg=1)
model.save('wonju.model')
print('ok')

ok


In [37]:
results

['컨테이너',
 '컨테이너 형',
 '컨테이너 형 가설',
 '컨테이너 형 가설 건축물',
 '컨테이너 형 가설 건축물',
 '컨테이너 형 가설 건축물 창고',
 '컨테이너 형 가설 건축물 창고',
 '컨테이너',
 '컨테이너 형',
 '컨테이너 형 가설',
 '컨테이너 형 가설 건축물',
 '컨테이너 형 가설 건축물',
 '컨테이너 형 가설 건축물 사무실',
 '컨테이너 형 가설 건축물 사무실',
 '평판',
 '평판 재다',
 '평판 재다 시험',
 '조립',
 '조립 식',
 '조립 식 가설',
 '조립 식 가설 실험실',
 '수평',
 '수평 규준',
 '수평 규준 틀',
 '구',
 '구 조부',
 '구 조부 먹매김',
 '강관',
 '강관 비계',
 '강관 비계',
 '강관 비계 쌍줄',
 '강관 비계 쌍줄',
 '강관 비계 쌍줄 설치',
 '강관 비계 쌍줄 설치 및',
 '강관 비계 쌍줄 설치 및 해체',
 '강관',
 '강관 조립',
 '강관 조립 말',
 '강관 조립 말 비계',
 '강관 조립 말 비계',
 '강관 조립 말 비계 이동식',
 '강관 조립 말 비계 이동식',
 '강',
 '강 관동',
 '강 관동 바리',
 '강 관동 바리 설치',
 '강 관동 바리 설치 및',
 '강 관동 바리 설치 및 해체',
 '시스템',
 '시스템 동바리',
 '시스템 동바리 설치',
 '시스템 동바리 설치 및',
 '시스템 동바리 설치 및 해체',
 '건축물',
 '건축물 보양',
 '건축물 보양',
 '건축물 보양 콘크리트',
 '건축물',
 '건축물 현장',
 '건축물 현장 정리',
 '터',
 '터 파기',
 '터 파기',
 '터 파기 토사',
 '되다',
 '되다 메우다',
 '되다 메우다',
 '되다 메우다 토사',
 '되다 메우다 토사',
 '되다 메우다 토사 두께',
 '되다 메우다 토사 두께 30',
 '되다 메우다 토사 두께 30 cm',
 '토사',
 '토사 운반',
 '토사 운반',
 '토사 운반 단지',
 '토사 운반 단지 외',


In [35]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('wonju.model')

In [41]:
results

['컨테이너',
 '컨테이너 형',
 '컨테이너 형 가설',
 '컨테이너 형 가설 건축물',
 '컨테이너 형 가설 건축물',
 '컨테이너 형 가설 건축물 창고',
 '컨테이너 형 가설 건축물 창고',
 '컨테이너',
 '컨테이너 형',
 '컨테이너 형 가설',
 '컨테이너 형 가설 건축물',
 '컨테이너 형 가설 건축물',
 '컨테이너 형 가설 건축물 사무실',
 '컨테이너 형 가설 건축물 사무실',
 '평판',
 '평판 재다',
 '평판 재다 시험',
 '조립',
 '조립 식',
 '조립 식 가설',
 '조립 식 가설 실험실',
 '수평',
 '수평 규준',
 '수평 규준 틀',
 '구',
 '구 조부',
 '구 조부 먹매김',
 '강관',
 '강관 비계',
 '강관 비계',
 '강관 비계 쌍줄',
 '강관 비계 쌍줄',
 '강관 비계 쌍줄 설치',
 '강관 비계 쌍줄 설치 및',
 '강관 비계 쌍줄 설치 및 해체',
 '강관',
 '강관 조립',
 '강관 조립 말',
 '강관 조립 말 비계',
 '강관 조립 말 비계',
 '강관 조립 말 비계 이동식',
 '강관 조립 말 비계 이동식',
 '강',
 '강 관동',
 '강 관동 바리',
 '강 관동 바리 설치',
 '강 관동 바리 설치 및',
 '강 관동 바리 설치 및 해체',
 '시스템',
 '시스템 동바리',
 '시스템 동바리 설치',
 '시스템 동바리 설치 및',
 '시스템 동바리 설치 및 해체',
 '건축물',
 '건축물 보양',
 '건축물 보양',
 '건축물 보양 콘크리트',
 '건축물',
 '건축물 현장',
 '건축물 현장 정리',
 '터',
 '터 파기',
 '터 파기',
 '터 파기 토사',
 '되다',
 '되다 메우다',
 '되다 메우다',
 '되다 메우다 토사',
 '되다 메우다 토사',
 '되다 메우다 토사 두께',
 '되다 메우다 토사 두께 30',
 '되다 메우다 토사 두께 30 cm',
 '토사',
 '토사 운반',
 '토사 운반',
 '토사 운반 단지',
 '토사 운반 단지 외',


In [40]:
model.most_similar(positive=['운반비'])

KeyError: "word '운반비' not in vocabulary"